In [ ]:
import sys
import os
from typing import Dict, Callable, Tuple, List

SCRIPT_PATH = "/home/yifanhou/git/PyriteML/scripts"
sys.path.append(os.path.join(SCRIPT_PATH, '../'))


import numpy as np
import torch
import time
import dill
import hydra
from torch.utils.data import DataLoader


from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.dataset.base_dataset import BaseImageDataset, BaseDataset
from diffusion_policy.workspace.train_diffusion_unet_image_workspace import TrainDiffusionUnetImageWorkspace

data_path = "/home/yifanhou/training_outputs/"
ckpt_path = data_path + "2025.03.05_21.53.36_stow_no_force_202_stow_80/checkpoints/latest.ckpt"

device = torch.device('cpu')

# load checkpoint
if not ckpt_path.endswith('.ckpt'):
    ckpt_path = os.path.join(ckpt_path, 'checkpoints', 'latest.ckpt')
payload = torch.load(open(ckpt_path, 'rb'), map_location='cpu', pickle_module=dill)

In [ ]:
import robotmq as rmq
import pickle
import time
import numpy as np
import numpy.typing as npt

server = rmq.RMQServer(
    server_name="test_rmq_server", server_endpoint="ipc:///tmp/feeds/0"
)
client = rmq.RMQClient(
    client_name="test_rmq_client", server_endpoint="ipc:///tmp/feeds/0"
)
print("Server and client created")

server.add_topic("test_checkpoints", 10)

# Serialize the checkpoint
start_time = time.time()
pickle_data = pickle.dumps(payload)
dump_end_time = time.time()
server.put_data("test_checkpoints", pickle_data)
send_end_time = time.time()
time.sleep(0.01)

retrieve_start_time = time.time()
retrieved_data, timestamp = client.peek_data(topic="test_checkpoints", order="latest", n=1)
retrieve_end_time = time.time()
received_data = pickle.loads(retrieved_data[0])


print(
    f"Data size: {len(pickle_data) / 1024**2:.3f}MB. dump: {dump_end_time - start_time:.4f}s, send: {send_end_time - dump_end_time: .4f}s, retrieve: {retrieve_end_time - retrieve_start_time:.4f}s, load: {time.time() - retrieve_end_time:.4f}s)"
)


# use the received payload
cfg = received_data['cfg']
print("dataset_path:", cfg.task.dataset.dataset_path)

cls = hydra.utils.get_class(cfg._target_)
workspace = cls(cfg)
workspace: BaseWorkspace
workspace.load_payload(received_data, exclude_keys=None, include_keys=None)

policy = workspace.model
if cfg.training.use_ema:
    policy = workspace.ema_model
policy.num_inference_steps = cfg.policy.num_inference_steps # DDIM inference iterations

policy.eval().to(device)
policy.reset()

# use normalizer saved in the policy
sparse_normalizer, dense_normalizer = policy.get_normalizer()

shape_meta = cfg.task.shape_meta

